<a href="https://colab.research.google.com/github/fernandanlisboa/technologies_clf/blob/main/genetic_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração e Instalação



*   Davi Costa
*   Fernanda Lisboa
*   João Felipe
*   Maria Antônia





In [ ]:
import os
from google.colab import drive
import random
import itertools
import math
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import make_scorer

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Deixar a somente fixa para saber por que uma alteração do código provocou uma melhora ou piora
# Ajuda a todas as vezes que você rodar encontrar os mesmos resultados
# É preciso fixar as duas pois uma afeta a outra
np.random.seed(42)
random.seed(42)

In [ ]:
drive_path = '/content/drive'
directory = f'{drive_path}/MyDrive/Dasafio 01 - Classificação/Entrega_Desafio_1'
#directory = f'{drive_path}/MyDrive/Dasafio 01 - Classificação/Entrega_Desafio_1'
csv_file = 'base_textos_pre_processada.csv'

In [ ]:
drive.mount(drive_path)

Mounted at /content/drive


In [ ]:
df = pd.read_csv(f"{directory}/{csv_file}", sep="|")

In [ ]:
df.head()

,classification,excerpt_lem
0,1,software define networking sdn
1,1,spdy experimental protocol multiplexe multiple...
2,0,aspdotnet signalr library aspdotnet developer ...
3,2,apache kafka publish subscribe messaging fram...
4,1,kubernete open source implementation container...


# 1- População Inicial

In [ ]:
def create_combination(size):
  return [random.choice([0, 1]) for _ in range(size)]


In [ ]:
def initialize_population(pop_size, num_params):
  combinations = [create_combination(num_params) for i in range(pop_size)]
  population = np.unique(combinations, axis=0)

  while population.shape[0] < pop_size:
    difference = pop_size - population.shape[0]
    complement = [create_combination(num_params) for i in range(difference)]
    population = np.concatenate((population, complement))
    population = np.unique(population, axis=0)

  return population

# 2- Avaliação de Aptidão

## Treinando Modelo

In [ ]:
X = df['excerpt_lem']
y = df['classification']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                  y,
                                                  test_size=0.10, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size=0.10, random_state=42)

# Permite unigramas e bigramas
# Os bigramas importam, pois se desconsiderá-los o modelo tem um resultado puor
ngram_range = (1,2)

# Este é o número mínimo de documentos que uma palavra deve estar presente para ser mantida como recurso.
min_df = 2

# Este é o número máximo de documentos em que uma palavra pode estar presente para ser mantida como recurso.
max_df = 1. #100%

# Considera apenas os tokens mais relevantes, ordenados pela frequência do termo em todo o documento
# Se for None, ele considera todos as features
max_features = None

# Se True, aplica escala sublinear ao termo de frequência (TF), ou seja, substitui tf com 1 + log(tf).
# Isso tem o efeito de diminuir o impacto de tokens muito frequentes.
sublinear_tf=True

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        lowercase=True,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        sublinear_tf=sublinear_tf)

features_train = tfidf.fit_transform(X_train).toarray()
features_train_df = pd.DataFrame(features_train)

labels_train = y_train
print("Train shape: ", features_train.shape)

features_val = tfidf.transform(X_val).toarray()
features_val_df = pd.DataFrame(features_val)
labels_val = y_val
print("Val shape: ", features_val.shape)


features_test = tfidf.transform(X_test).toarray()
features_test_df = pd.DataFrame(features_test)
print("Test shape: ", features_test.shape)

Train shape:  (216, 801)
Val shape:  (24, 801)
Test shape:  (27, 801)


In [ ]:
def get_model(clf_name, hyperparams=None):
  model = None
  if clf_name == 'Random Forest':
    if hyperparams != None:
      model = RandomForestClassifier(**hyperparams, random_state=random_state)
    else:
      model = RandomForestClassifier(random_state=random_state)
  elif clf_name == 'Gradient Boosting':
    if hyperparams != None:
      model = GradientBoostingClassifier(**hyperparams, random_state=random_state)
    else:
      model = GradientBoostingClassifier(random_state=random_state)
  elif clf_name == 'Decision Tree':
      if hyperparams != None:
        model = DecisionTreeClassifier(**hyperparams, random_state=random_state)
      else:
        model = DecisionTreeClassifier(random_state=random_state)
  elif clf_name == 'Hist Gradient Boosting':
      if hyperparams != None:
        model = HistGradientBoostingClassifier(**hyperparams, random_state=random_state)
      else:
        model = HistGradientBoostingClassifier(random_state=random_state)

  return model

In [ ]:
def train_model(feat_train, feat_test):
  clf = get_model(model_name)
  clf.fit(feat_train, labels_train)
  y_pred = clf.predict(feat_test)
  # multiclass problem
  # macro: Calculate metrics for each label, and find their unweighted mean.
  # This does not take label imbalance into account
  # The three classes are balanced in the complete dataset
  f1 = f1_score(labels_val, y_pred, average='macro')

  return math.ceil(f1 * 100)

## Calculando Score


In [ ]:
# Retira as colunas de um dataset com base em um array de 0 e 1
def configure_df(params, df):
  columns = df.columns
  colums = [columns[i] for i in range(len(columns)) if params[i] == 0]
  df = df.drop(colums, axis=1)
  return df

In [ ]:
def calculate_params_score(params):
  maxNumAttributes = math.ceil(len(params)*0.6) #número máximo de atributos é 60% do total
  minNumAttributes = math.ceil(len(params)*0.2) #número mínimo de atributos é 20% do total

  #penalizando indivíduos que não tem o número mínimo de atributos e indivíduos que tem mais atributos que o máximo desejado
  if np.count_nonzero(params == 1) > maxNumAttributes or np.count_nonzero(params == 1) < minNumAttributes:
    score = 0
  else:
    feat_train = configure_df(params, features_train_df)
    feat_test = configure_df(params, features_val_df)
    score = train_model(feat_train, feat_test)

  return score

# 3- Seleção

In [ ]:
def select_parents(population, parents_rate=0.2):
  fitness_scores = np.apply_along_axis(calculate_params_score, axis=1, arr=population)

  selected_indices = np.random.choice(len(population), size=int(parents_rate*len(population)),
                                        p=fitness_scores / sum(fitness_scores))

  return [population[i] for i in selected_indices]

# 4- Cruzamento

In [ ]:
def crossover(parent1, parent2):
    # Escolha aleatória do ponto de crossover
    # Selecionando o ponto, sendo que não pode ser o primeiro nem o último para não ficar igual a um pai
    # Randit tem intervalo aberto, então não está considerando o último
    crossover_point = random.randint(1, len(parent1) - 1)

    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

# 5- Mutação

In [ ]:
# Existe uma probabilidade de mutação, pois a mutação não é aplicada em todos
# Também existe um percentual de atributos que serão invertidos
def mutate(params, mutation_rate = 100, inversion_rate = 0.3):
    if random.random() < mutation_rate:
        params_list = list(params)
        # Número de atributos que serão alterados
        numInversion = math.ceil(len(params_list)*inversion_rate)
        # Números aleatórios entre as posições do seu vetor indivíduo
        indexList = random.sample(range(len(params_list)), numInversion)
        # Loop para inverter o valor dos atributos
        for index in indexList:
          if(params_list[index] == 0):
            params_list[index] = 1
          else:
            params_list[index] = 0
        return tuple(params_list)
    else:
      return params


# Execução do Algoritmo Genético


In [ ]:
# Parâmetros
population_size = 1000
generations = 50
num_params = pd.DataFrame(features_train).shape[1] #Número total de parâmetros
mutation_rate = 0.1  # Define a porcentagem de mutação
inversion_rate = 0.2 # Define a porcentagem de parâmetros que terão seu valor invertido na mutação
parents_rate = 0.2   # Define a porcentagem de pais
model_name = 'Decision Tree' # Modelo utilizado na seleção
random_state = 42

In [ ]:
population = initialize_population(population_size, num_params)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import random

def generate_offspring(parents_tuple):
    parent1, parent2 = parents_tuple
    child1, child2 = crossover(parent1, parent2)
    return [mutate(child1, mutation_rate, inversion_rate), mutate(child2, mutation_rate, inversion_rate)]

In [ ]:
%%time
for generation in range(generations):
    print(f"-- {generation=}")
    parents = select_parents(population, parents_rate)
    offspring = []

    with ThreadPoolExecutor() as executor:
        parents_tuples = [(parent, random.choice(parents)) for parent in parents]
        offspring_lists = list(executor.map(generate_offspring, parents_tuples))
        for o in offspring_lists:
            offspring.extend(o)

    combined_population = np.unique(np.vstack((parents, offspring)), axis=0)

    with ThreadPoolExecutor() as executor:
        fitness_scores = list(executor.map(calculate_params_score, combined_population))

    sorted_indices = np.argsort(fitness_scores)[::-1]
    sorted_indices = sorted_indices[(sorted_indices < combined_population.shape[0])]
    combined_population = combined_population[sorted_indices[:population_size]]

    while len(combined_population) < population_size:
        random_individual = np.copy(random.choice(combined_population))
        mutated_individual = mutate(random_individual, 1, inversion_rate)
        if not np.any(np.all(combined_population == mutated_individual, axis=1)):
            combined_population = np.unique(np.vstack((combined_population, mutated_individual)), axis=0)

    population = combined_population[:population_size]

best_params = max(population, key=calculate_params_score)
print(f"Melhores parâmetros: {best_params} \nScore: {int(calculate_params_score(best_params))}")


-- generation=0
-- generation=1
-- generation=2
-- generation=3
-- generation=4
-- generation=5
-- generation=6
-- generation=7
-- generation=8
-- generation=9
-- generation=10
-- generation=11
-- generation=12
-- generation=13
-- generation=14
-- generation=15
-- generation=16
-- generation=17
-- generation=18
-- generation=19
-- generation=20
-- generation=21
-- generation=22
-- generation=23


# Attribute Selection

In [ ]:
best_val_df = configure_df(best_params, features_val_df)
best_train_df = configure_df(best_params, features_train_df)
best_test_df = configure_df(best_params, features_test_df)

In [ ]:
genetic_f1 = train_model(best_train_df, best_val_df)
genetic_f1

In [ ]:
clf = get_model(model_name)
clf.fit(best_train_df, y_train)

pred_genetic = clf.predict(best_test_df)
f1_score(y_test, pred_genetic, average='macro')

# Kbest

In [ ]:
features_value = len(best_train_df.columns)
features_value

In [ ]:
selector = SelectKBest(chi2, k=features_value) # k de acordo com a quantidade de atributos selecionada pelo algoritmo genético
features_train_kbest = selector.fit_transform(features_train, labels_train)
features_test_kbest = selector.transform(features_test)
features_val_kbest = selector.transform(features_val)

In [ ]:
kbest_f1 = train_model(features_train_kbest, features_val_kbest)
kbest_f1

In [ ]:
clf = get_model(model_name)
clf.fit(features_train_kbest, y_train)

pred_kbest = clf.predict(features_test_kbest)
f1_score(y_test, pred_kbest, average='macro')

# RandomSearch

In [ ]:
def get_params(model_name):
    depths = [None]

    params = dict()
    if model_name == 'Decision Tree':
        depths.extend(np.arange(1, 20, 1, dtype=int).tolist())
        params = {
                    'max_depth': depths,
                    'min_samples_split': np.arange(2, 10, 1, dtype=int),
                    'min_samples_leaf': np.arange(1, 10, 1, dtype=int)
                }
    elif model_name == 'Random Forest':
        depths.extend(np.arange(20, 200, 10, dtype=int).tolist())
        params = {
              'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)],
              'max_depth': depths,
              'min_samples_split': np.arange(2, 20, 1, dtype=int),
              'min_samples_leaf': np.arange(1, 20, 1, dtype=int),
              'max_features': ['auto', 'sqrt'],
              'bootstrap':  [True, False]
          }

    elif model_name == 'Gradient Boosting':
        depths.extend(np.arange(1, 5, 1, dtype=int).tolist())
        params = {
                        'max_depth': depths,
                        'min_samples_split': np.arange(0.01, 0.1, 0.01, dtype=float),
                        'learning_rate': np.arange(0.01, 0.1, 0.01, dtype=float)
                    }
    elif model_name == 'Hist Gradient Boosting':
        depths.extend(np.arange(1, 5, 1, dtype=int).tolist())
        params = {
                        'max_depth': depths,
                        'min_samples_leaf': [1,5,10,15,20,25],
                        'learning_rate': np.arange(0.01, 0.1, 0.01, dtype=float)
                    }


    return params


In [ ]:
# o randomsearch só será feito ao final, após a seleção dos MELHORES atributos
# TODO add others classifiers, like SVM
def random_search(clf_name, ft_train, labels):
  score_metrics = {'F1': 'f1_macro', 'Accuracy': 'accuracy'}
  model = get_model(clf_name)
  clf_params = get_params(clf_name)

  random_search = RandomizedSearchCV(estimator=model,
                                   param_distributions=clf_params,
                                   n_iter=10,
                                   scoring=score_metrics,
                                   refit='F1',
                                   cv=5,
                                   verbose=1,
                                   n_jobs = -1)


  random_search.fit(ft_train, labels)

  print(random_search.cv_results_)

  return random_search.best_params_

In [ ]:
# executando os melhores atributos no grid search

# algoritmo genético
best_clf_params = random_search(model_name, best_train_df, labels_train)

print(f"Best {model_name}'s params:\n{best_clf_params}")

In [ ]:
best_clf_params

In [ ]:

best_clf_params_kbest = random_search(model_name, features_train_kbest, labels_train)

print(f"Best {model_name}'s params:\n{best_clf_params_kbest}")

# Training with only best params

In [ ]:
# usando os parametros do algoritmo genético
final_model = get_model(model_name, best_clf_params)
final_model

In [ ]:
final_model.fit(best_train_df, labels_train)

In [ ]:
final_model_kbest = get_model(model_name, best_clf_params_kbest)
final_model_kbest

In [ ]:
final_model_kbest.fit(features_train_kbest, labels_train)

## Final score

In [ ]:
y_pred = final_model.predict(best_test_df)
y_pred_kbest = final_model_kbest.predict(features_test_kbest)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

acc_kbest = accuracy_score(y_test, y_pred_kbest)
f1_kbest = f1_score(y_test, y_pred_kbest, average='macro')

print('Algoritmo genético: ')
print(f"Accuracy: {acc}")
print(f"F1 score: {f1}")
print('Algoritmo KBest: ')
print(f"Accuracy: {acc_kbest}")
print(f"F1 score: {f1_kbest}")

In [ ]:
def saving_results(clf_name, f1_value, num_att, att_selection, full_path=''):

    csv_path = os.path.join(full_path, 'results.csv')

    dict_data = {'model_name': [clf_name],
                 'features': [num_att],
                 'attribute_selection': [att_selection],
                 'f1_score': [f1_value]
                 }
    # checking if we've already had some csv file result
    if os.path.exists(csv_path):
        df_res = pd.read_csv(csv_path, sep=';')
        df_ = pd.DataFrame(dict_data)
        df_res = pd.concat([df_res, df_])
    else:
       df_res = pd.DataFrame(dict_data)

    df_res.to_csv(csv_path, sep=';', index=False)

    print(f"CSV file saved with succcess!")

    return df_res

In [ ]:
# clf_name, f1_value, num_att, att_selection, full_path=''
saving_results(model_name, f1, features_value, 'Genetico', directory)

In [ ]:
saving_results(model_name, f1_kbest, features_value, 'Kbest', directory)

# Avaliando modelos gerados

## Primeiro Treino

O primeiro treino foi feito com `min_df=6` no TFIDF, `n_iter=5` no `RandomizedSearchCV` e 50 gerações com 500 de população no algoritmo genético.

In [ ]:
pd.read_csv(f"{directory}/results/old_result_50gen_500pop.csv", sep=";")

,model_name,features,attribute_selection,f1_score
0,Decision Tree,65,Genetico,0.489815
1,Decision Tree,65,Kbest,0.550446
2,Gradient Boosting,62,Genetico,0.548203
3,Gradient Boosting,62,Kbest,0.572264
4,Hist Gradient Boosting,61,Genetico,0.541533
5,Hist Gradient Boosting,61,Kbest,0.509524
6,Random Forest,72,Genetico,0.472339
7,Random Forest,72,Kbest,0.592157


Observando os resultados obtidos e analisando os modelos treinados observando apenas a seleção de atributos do KBest, percebe-se que o classificador Random Forest, consegue a melhor performance, com 59,21% de F1 score. Enquanto, para o Algoritmo genético, o maior valor foi alcançado pelo Gradient Boosting, com 54,82% de F1 score.

Vale ressaltar, que ao executar o treinamento dos modelos sem alteração dos valores de seus parâmetros, usando apenas os conjuntos de treino e validação, percebemos que em todos os casos usando o algoritmo do KBest obteve performance menor do que utilizando o Genético. Isso nos faz questionar sobre as caraceterísticas dos nossos conjuntos de dados, já que, mesmo passando pelo RandomSearchCV, a performance decaiu para usando ambos algoritmos de seleção de atributos, equanto esperava-se que o desempenho fosse melhorado com o uso dos dois algoritmos.

Sendo assim, é interessante investigar quais características dos nossos conjuntos de treino e validação faltaram para que o modelo fosse um pouco mais generalizado, além do que mais o conjunto de teste poderia estar constituído de diferente dos de treino e validação.

Além disso, percebeu-se que o algoritmo genético apresentou um score F1 maior que o KBest quanto utilizamos o modelo Hist Gradient Boosting.

Acreditamos, que outra hipótese para a diminuição do score do algoritmos genético após o RandomSearch é que a nossa seleção de atributos pode ter sido enviesada pelo conjunto de validação do modelo.

Somado a isso, observamos que no modelo Random Forest, o algoritmo genético resultou na maior quantidade de atributos selecionados e, coincidentemente, foi o classificador, que mesmo após a aplicação do Random Search CV para encontrar os melhores parâmetros, obteve pior performance comparado aos outros modelos usando os atributos selecionados pelo algoritmo genético.

## Segundo Treino

Alteramos o parâmetro `min_df` do TFIDF de 6 para 2, além de seguir as recomendações da documentação do sklearn modificando os parâmetros `n_inter` do `RandomizedSearchCV` para 10, respectivamente, sendo esses os melhores parâmetros encontrados para os novos testes. Além disso, o algoritmo genético rodou 5 gerações com 100 de população.


In [ ]:
pd.read_csv(f"{directory}/results/result_5gen_100pop.csv", sep=";")

,model_name,features,attribute_selection,f1_score
0,Random Forest,388,Genetico,0.585185
1,Random Forest,388,Kbest,0.551852
2,Gradient Boosting,381,Genetico,0.508772
3,Gradient Boosting,381,Kbest,0.630556
4,Decision Tree,418,Genetico,0.334392
5,Decision Tree,418,Kbest,0.441234
6,Hist Gradient Boosting,397,Genetico,0.558881
7,Hist Gradient Boosting,397,Kbest,0.671053


A partir dos novos resultados, percebeu-se que a quantidade de _features_ aumentou significativamente, isso aconteceu pois diminuimos a quantidade de documentos necessários em que uma palavra precisa estar para ser considerada relevante no TFIDF, então mais _tokens_ foram adicionados aos treinos. Isso permitiu que nosso modelo utilizasse mais palavras para realizar a classificação, o que melhorou levemente as pontuações. No entanto, apenas o `RandomForest` foi melhor que o KBest.

As alterações realizadas no `RandomizedSearchCV` diminuiram a alta diferença entre o resultado do algoritmo genético e o resultado final, pois com o aumento do número de iterações no CV, pôde-se treinar mais vezes com cada combinação de hiperparâmetros passada, possibilitando encontrar uma melhor combinação e valores mais altos de `score`.

Vale ressaltar também que a variação de scores obtidos entre as execuções do `RandomSearchCV` influenciam na média encontrada, uma vez que, dependendo da amostra de dados executada no algoritmo, o valor de `score` obtido pode ser tão alto, que aumente drasticamente essa média resultante. O que também pode acontecer é do `score` ser tão baixo que reduza bastante a média obtida, considerando apenas para uma amostra específica com uma combinação de parâmetros exclusiva, não significando que aquela média realmente representa os resultados adquiridos. Dessa forma, é interessante realizar uma investigação acerca dessas amostras com resultados "fora do comum", para saber quais características esses dados possuem para impactar com valores tão diferentes do que foi obtido em outras amostras.